![CoSAppLogo](images/cosapp.svg) **CoSApp** tutorials: Simulation logging system

# Simulation log

CoSApp simulation logger is built on top of the Python built-in package `logging`. To set the simulation log, you will use the helper function `set_log`.


In [ ]:
from cosapp.utils import LogLevel, set_log

help(set_log)

## Example

The following cell creates the two tanks problem (see [Transient simulations](TimeDriver.ipynb) tutorial for more information). The usage of `set_log` will be demonstrated on it.

In [ ]:
import numpy as np
from cosapp.base import System, Port
from cosapp.drivers import EulerExplicit, NonLinearSolver


class FloatPort(Port):
    def setup(self):
        self.add_variable('value', 0.0)


class Tank(System):
    def setup(self, rho=1e3):
        self.add_inward('area', 1.0, desc='Cross-section area')
        self.add_inward('rho', abs(rho), desc='Fluid density')

        self.add_input(FloatPort, 'flowrate')
        self.add_output(FloatPort, 'p_bottom')

        self.add_transient('height', der='flowrate.value / area')

    def compute(self):
        g = 9.81
        self.p_bottom.value = self.rho * g * self.height


class Pipe(System):
    """Poiseuille flow in a cylindrical pipe"""
    def setup(self):
        self.add_inward('D', 0.1, desc="Diameter")
        self.add_inward('L', 2.0, desc="Length")
        self.add_inward('mu', 1e-3, desc="Fluid dynamic viscosity")

        self.add_input(FloatPort, 'p1')
        self.add_input(FloatPort, 'p2')

        self.add_output(FloatPort, 'Q1')
        self.add_output(FloatPort, 'Q2')

        self.add_outward('k', desc='Pressure loss coefficient')

    def compute(self):
        """Computes the volumetric flowrate from the pressure drop"""
        self.k = np.pi * self.D**4 / (256 * self.mu * self.L)
        self.Q1.value = self.k * (self.p2.value - self.p1.value)
        self.Q2.value = -self.Q1.value


class CoupledTanks(System):
    """System describing two tanks connected by a pipe (viscous limit)"""
    def setup(self, rho=1e3):
        self.add_child(Tank('tank1', rho=rho))
        self.add_child(Tank('tank2', rho=rho))
        self.add_child(Pipe('pipe'))

        self.connect(self.tank1.p_bottom, self.pipe.p1)
        self.connect(self.tank2.p_bottom, self.pipe.p2)
        self.connect(self.tank1.flowrate, self.pipe.Q1)
        self.connect(self.tank2.flowrate, self.pipe.Q2)


## `set_log` usage

### Default usage

In its default usage, the simulation log will be stored in a file with the default name in the current folder. Only messages of level `LogLevel.INFO` or above will be displayed.

In [ ]:
set_log()

system = CoupledTanks("coupledTanks", rho=1e3)
driver = system.add_driver(EulerExplicit(dt=0.1, time_interval=[0, 0.1]))

solver = driver.add_child(NonLinearSolver("solver", factor=1.0))

h1_0, h2_0 = (3, 1)

driver.set_scenario(
    name="run",
    init={"tank1.height": h1_0, "tank2.height": h2_0,},  # initial conditions
    values={"pipe.D": 0.07, "pipe.L": 2.5, "tank1.area": 2,},  # fixed values
)

system.run_drivers()

### Set log level

If you want to have more of less information, you could set the level of message to be recorded according to the following scale:

In [ ]:
LogLevel?

In [ ]:
set_log(level=LogLevel.WARNING) # less information (none in this case)

system = CoupledTanks("coupledTanks", rho=1e3)
driver = system.add_driver(EulerExplicit(dt=0.1, time_interval=[0, 0.1]))

solver = driver.add_child(NonLinearSolver("solver", factor=1.0))

h1_0, h2_0 = (3, 1)

driver.set_scenario(
    name="run",
    init={"tank1.height": h1_0, "tank2.height": h2_0,},  # initial conditions
    values={"pipe.D": 0.07, "pipe.L": 2.5, "tank1.area": 2,},  # fixed values
)

system.run_drivers()  # Should no emit any log message

In [ ]:
set_log(level=LogLevel.DEBUG) # more information

system = CoupledTanks("coupledTanks", rho=1e3)
driver = system.add_driver(EulerExplicit(dt=0.1, time_interval=[0, 0.1]))

solver = driver.add_child(NonLinearSolver("solver", factor=1.0))

h1_0, h2_0 = (3, 1)

driver.set_scenario(
    name="run",
    init={"tank1.height": h1_0, "tank2.height": h2_0,},  # initial conditions
    values={"pipe.D": 0.07, "pipe.L": 2.5, "tank1.area": 2,},  # fixed values
)

system.run_drivers()

### Add time filter

With level lower than `DEBUG`, the number of message recorded may become large. Usually, though, when debugging, you may be interested in debug messages emitted after a given simulation time, say. You can achieve that by specifying the `start_time` keyword:

In [ ]:
set_log(level=LogLevel.DEBUG, start_time=0.1)  # Be more verbose for time greater than 0.1s

system = CoupledTanks("coupledTanks", rho=1e3)
driver = system.add_driver(EulerExplicit(dt=0.1, time_interval=[0, 0.1]))

solver = driver.add_child(NonLinearSolver("solver", factor=1.0))

h1_0, h2_0 = (3, 1)

driver.set_scenario(
    name="run",
    init={"tank1.height": h1_0, "tank2.height": h2_0,},  # initial conditions
    values={"pipe.D": 0.07, "pipe.L": 2.5, "tank1.area": 2,},  # fixed values
)

system.run_drivers()

### Add context filter

Another possibility would be to filter by context, i.e. by systems or by driver.

> When filtering by system, the prescribed system *and all its children* will be part of the more verbose log. See for example the log for the dummy subsystem `subtank1` in the following example.

In [ ]:
set_log(level=LogLevel.DEBUG, context="tank1") # Filter on a System

system = CoupledTanks("coupledTanks", rho=1e3)
driver = system.add_driver(EulerExplicit(dt=0.1, time_interval=[0, 0.1]))

system.tank1.add_child(System("subtank1"))
solver = driver.add_child(NonLinearSolver("solver", factor=1.0))

h1_0, h2_0 = (3, 1)

driver.set_scenario(
    name="run",
    init={"tank1.height": h1_0, "tank2.height": h2_0,},  # initial conditions
    values={"pipe.D": 0.07, "pipe.L": 2.5, "tank1.area": 2,},  # fixed values
)

system.run_drivers()

In [ ]:
set_log(level=LogLevel.DEBUG, context="solver") # Filter on a Driver

system = CoupledTanks("coupledTanks", rho=1e3)
driver = system.add_driver(EulerExplicit(dt=0.1, time_interval=[0, 0.1]))

solver = driver.add_child(NonLinearSolver("solver", factor=1.0))

h1_0, h2_0 = (3, 1)

driver.set_scenario(
    name="run",
    init={"tank1.height": h1_0, "tank2.height": h2_0,},  # initial conditions
    values={"pipe.D": 0.07, "pipe.L": 2.5, "tank1.area": 2,},  # fixed values
)

system.run_drivers()

## `FULL_DEBUG` mode

The `DEBUG` mode provides a detailed trace of calls done by the solver within the simulation system. When bad comes to worse, though, nothing is more valuable than values to analyze a simulation. This is the purpose of the `FULL_DEBUG` mode. Its behavior will depend on the object. What you could expect for now is:

- For all `System` objects: Record the values of inputs (including inwards) and outputs (including outwards)
- For `NonLinearSolver`: Record the computed Jacobian matrix and the evolution of the unknowns and the residues. These data are printed as comma-separated tables to ease their post-processing, such as plotting a graph of their evolution, e.g.
- For `TimeDriver` objects: Record actual simulation time steps.

### Records for `System`

In [ ]:
set_log(level=LogLevel.FULL_DEBUG, context="tank1", start_time=0.1)

system = CoupledTanks("coupledTanks", rho=1e3)
driver = system.add_driver(EulerExplicit(dt=0.1, time_interval=[0, 0.1]))

solver = driver.add_child(NonLinearSolver("solver", factor=1.0))

h1_0, h2_0 = (3, 1)

driver.set_scenario(
    name="run",
    init={"tank1.height": h1_0, "tank2.height": h2_0,},  # initial conditions
    values={"pipe.D": 0.07, "pipe.L": 2.5, "tank1.area": 2,},  # fixed values
)

system.run_drivers()

### Records for `NonLinearSolver`

In [ ]:
set_log(level=LogLevel.FULL_DEBUG, context="solver", start_time=0.1)

system = CoupledTanks("coupledTanks", rho=1e3)
driver = system.add_driver(EulerExplicit(dt=0.1, time_interval=[0, 0.1]))

solver = driver.add_child(NonLinearSolver("solver", factor=1.0))

h1_0, h2_0 = (3, 1)

driver.set_scenario(
    name="run",
    init={"tank1.height": h1_0, "tank2.height": h2_0,},  # initial conditions
    values={"pipe.D": 0.07, "pipe.L": 2.5, "tank1.area": 2,},  # fixed values
)

system.run_drivers()

### Records for `TimeDriver`

In [ ]:
set_log(level=LogLevel.FULL_DEBUG, context="euler")

system = CoupledTanks("coupledTanks", rho=1e3)
driver = system.add_driver(EulerExplicit("euler", dt=0.1, time_interval=[0, 0.2]))

solver = driver.add_child(NonLinearSolver("solver", factor=1.0))

h1_0, h2_0 = (3, 1)

driver.set_scenario(
    name="run",
    init={"tank1.height": h1_0, "tank2.height": h2_0,},  # initial conditions
    values={"pipe.D": 0.07, "pipe.L": 2.5, "tank1.area": 2,},  # fixed values
)

system.run_drivers()